# Toronto Neighborhoods DataFrame

In [5]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [23]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text   # obtain source from Wikipedia page
soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())

table = soup.find('table',class_='wikitable')   # locate specific table on Wikipedia page

#for data in table.find_all('tr'):
    #data = data.text
    #print(data)

df = pd.read_html(str(table))
toronto = pd.DataFrame(df[0])   # convert table into dataframe
toronto.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [31]:
toronto.drop(toronto.loc[toronto['Borough']=='Not assigned'].index,inplace=True)   # remove rows where the "Borough" column has a "Not assigned" value
toronto.reset_index(drop=True,inplace=True)   # reset the index
toronto = toronto.rename({'Postal code': 'PostalCode'}, axis=1)   # rename the "Postal code" column
toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [48]:
print('Not assigned' in toronto['Neighborhood'])   # check if there are "not assigned" values in the "Neighborhood" column
print(toronto['PostalCode'].duplicated().any())   # check if there are duplicate values in the "PostalCode" column

False
False


In [35]:
toronto.shape   # check number of rows and columns in dataframe

(103, 3)

# Toronto Neighborhood Coordinates

In [42]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')   # obtain coordinates from csv file
coordinates = coordinates.rename({'Postal Code': 'PostalCode'}, axis=1)   # rename the "Postal code" column
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [49]:
toronto_coord = toronto.merge(coordinates, on="PostalCode", how="left")   # add "Latitude" and "Longitude" data from "Coordinates" table to "Toronto" table
toronto_coord.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [47]:
print('NaN' in toronto_coord['Latitude'])   # check if there are "NaN" values in the "Latitude" column
print('NaN' in toronto_coord['Longitude'])   # check if there are "NaN" values in the "Longitude" column

False
False


# Toronto Neighborhood Clusters

In [56]:
!pip install geopy
!pip install folium

     |████████████████████████████████| 92kB 7.9MB/s eta 0:00:011


In [57]:
!conda install -c anaconda xlrd --yes
!conda update -n base -c defaults conda --yes

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs:
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xlrd-1.2.0                 |           py36_0         188 KB  anaconda
    ------------------------------------------------------------
                                           Total:         188 KB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2019.11.~ --> anaconda::ca-certificates-2020.1.1-0
  openssl            conda-forge::openssl-1.1.1f-h516909a_0 --> anaconda::openssl-1.1.1-h7b6447c_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2019.11.28-py36h~ --> anaconda::certifi-2019.11.28-py36_1
  xlrd                     anaconda/noarch::xlrd-1.2.0-py_0 --> anaconda/linux-64::xlrd-1.2.0-py36_0

In [58]:
# obtain Toronto coordinates

address = 'toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [65]:
# create map of Toronto using coordinates
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for post, borough, neighborhood, lat, lng in zip(toronto_coord['PostalCode'], toronto_coord['Borough'], toronto_coord['Neighborhood'], toronto_coord['Latitude'], toronto_coord['Longitude']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [131]:
# define Foursquare Credentials and Version
CLIENT_ID = 'xxx'
CLIENT_SECRET = 'xxx'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxx
CLIENT_SECRET:xxx


In [75]:
# identify venues in each Toronto neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [76]:
# Create new dataframe of Toronto venues
toronto_venues = getNearbyVenues(names=toronto_coord['Neighborhood'],
                                   latitudes=toronto_coord['Latitude'],
                                   longitudes=toronto_coord['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [78]:
print(toronto_venues.shape)
toronto_venues.head(20)

(684, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,Regent Park / Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,Regent Park / Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,Regent Park / Harbourfront,43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [79]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Alderwood / Long Branch,8,8,8,8,8,8
Bathurst Manor / Wilson Heights / Downsview North,10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,10,10,10,10,10,10
Business reply mail Processing CentrE,10,10,10,10,10,10


In [80]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 182 uniques categories.


In [88]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = list(toronto_onehot)
# move the "Neighborhood" column to head of list
fixed_columns.insert(0, fixed_columns.pop(fixed_columns.index('Neighborhood')))
# reorder columns
toronto_onehot = toronto_onehot.loc[:, fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [89]:
toronto_onehot.shape

(684, 182)

In [91]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0000,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.2000,0.0,0.000000,0.0,0.00,0.0,0.0,0.0000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.0000,0.0000,0.00,0.000000,0.0,0.2,0.000000,0.000000,0.2,0.00,0.0,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.00,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.0000,0.0,0.00,0.000,0.0000,0.000000,0.0000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.200000,0.000000,0.000,0.000000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.00,0.00,0.0,0.00,0.000,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
1,Alderwood / Long Branch,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.0000,0.0,0.125000,0.0,0.00,0.0,0.0,0.0000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0000

In [92]:
toronto_grouped.shape

(93, 182)

In [93]:
# Top 5 common venues per neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1               Skating Rink   0.2
2                     Lounge   0.2
3  Latin American Restaurant   0.2
4             Clothing Store   0.2


----Alderwood / Long Branch----
          venue  freq
0   Pizza Place  0.25
1      Pharmacy  0.12
2           Gym  0.12
3  Skating Rink  0.12
4           Pub  0.12


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                Coffee Shop   0.2
1                Bridal Shop   0.1
2  Middle Eastern Restaurant   0.1
3           Sushi Restaurant   0.1
4             Sandwich Place   0.1


----Bayview Village----
                      venue  freq
0       Japanese Restaurant  0.25
1                      Café  0.25
2        Chinese Restaurant  0.25
3                      Bank  0.25
4  Mediterranean Restaurant  0.00


----Bedford Park / Lawrence Manor East----
                     venue  freq
0              Coffee Shop  

In [118]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [119]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Skating Rink,Latin American Restaurant,Clothing Store,Lounge,Dog Run,Fast Food Restaurant,Farmers Market,Event Space,Empanada Restaurant
1,Alderwood / Long Branch,Pizza Place,Pharmacy,Skating Rink,Gym,Coffee Shop,Sandwich Place,Pub,Dessert Shop,Eastern European Restaurant,Drugstore
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Middle Eastern Restaurant,Restaurant,Sandwich Place,Deli / Bodega,Sushi Restaurant,Ice Cream Shop,Bridal Shop,Bank,Event Space
3,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Field,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant
4,Bedford Park / Lawrence Manor East,Coffee Shop,Thai Restaurant,Comfort Food Restaurant,Café,Restaurant,Italian Restaurant,Pub,Sushi Restaurant,Indian Restaurant,Discount Store


In [121]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 1, 4, 1, 1, 4, 1, 1], dtype=int32)

In [122]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_coord

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Convenience Store,Park,Food & Drink Shop,Discount Store,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1,Breakfast Spot,Park,Coffee Shop,Spa,Distribution Center,Restaurant,Pub,Bakery,Historic Site,Curling Ice
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,1,Clothing Store,Accessories Store,Women's Store,Event Space,Boutique,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Furniture / Home Store,Distribution Center
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,4,Coffee Shop,Yoga Studio,Arts & Crafts Store,Creperie,Distribution Center,Beer Bar,Sushi Restaurant,Italian Restaurant,Park,Gastropub


In [123]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [124]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,Fast Food Restaurant,Field,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center


In [125]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Convenience Store,Park,Food & Drink Shop,Discount Store,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
2,Downtown Toronto,1,Breakfast Spot,Park,Coffee Shop,Spa,Distribution Center,Restaurant,Pub,Bakery,Historic Site,Curling Ice
3,North York,1,Clothing Store,Accessories Store,Women's Store,Event Space,Boutique,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Furniture / Home Store,Distribution Center
7,North York,1,Japanese Restaurant,Gym / Fitness Center,Discount Store,Café,Restaurant,Caribbean Restaurant,Sporting Goods Shop,Bike Shop,Italian Restaurant,Baseball Field
13,North York,1,Japanese Restaurant,Gym / Fitness Center,Discount Store,Café,Restaurant,Caribbean Restaurant,Sporting Goods Shop,Bike Shop,Italian Restaurant,Baseball Field
8,East York,1,Pizza Place,Pharmacy,Bank,Gastropub,Intersection,Pet Store,Café,Bus Line,Gym / Fitness Center,Asian Restaurant
9,Downtown Toronto,1,Ramen Restaurant,Theater,Comic Shop,Music Venue,Tea Room,Burrito Place,Pizza Place,Café,Clothing Store,Plaza
10,North York,1,Park,Pizza Place,Pub,Japanese Restaurant,Yoga Studio,Diner,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
12,Scarborough,1,History Museum,Bar,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant
14,East York,1,Pharmacy,Beer Store,Cosmetics Shop,Curling Ice,Diner,Skating Rink,Park,Spa,Video Store,Gastropub


In [128]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,2,Baseball Field,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
101,Etobicoke,2,Construction & Landscaping,Deli / Bodega,Baseball Field,Distribution Center,Fast Food Restaurant,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant


In [129]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,3,Park,Women's Store,Market,Yoga Studio,Discount Store,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant
32,Scarborough,3,Grocery Store,Playground,Yoga Studio,Diner,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run
85,Scarborough,3,Park,Playground,Yoga Studio,Diner,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run
91,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Diner,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore


In [130]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
4,Downtown Toronto,4,Coffee Shop,Yoga Studio,Arts & Crafts Store,Creperie,Distribution Center,Beer Bar,Sushi Restaurant,Italian Restaurant,Park,Gastropub
22,Scarborough,4,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Farmers Market,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
24,Downtown Toronto,4,Coffee Shop,Park,Japanese Restaurant,Sushi Restaurant,Italian Restaurant,Gastropub,Modern European Restaurant,Curling Ice,Dance Studio,Empanada Restaurant
28,North York,4,Coffee Shop,Middle Eastern Restaurant,Restaurant,Sandwich Place,Deli / Bodega,Sushi Restaurant,Ice Cream Shop,Bridal Shop,Bank,Event Space
34,North York,4,Caribbean Restaurant,Massage Studio,Coffee Shop,Miscellaneous Shop,Bar,Yoga Studio,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant
38,Scarborough,4,Chinese Restaurant,Hobby Shop,Department Store,Coffee Shop,Convenience Store,Dance Studio,Dog Run,Fast Food Restaurant,Farmers Market,Event Space
42,Downtown Toronto,4,Coffee Shop,Café,Tea Room,Gym,Beer Bar,Hotel,Pub,Restaurant,Yoga Studio,Empanada Restaurant
43,West Toronto,4,Coffee Shop,Gym,Furniture / Home Store,Climbing Gym,Café,Breakfast Spot,Bar,Italian Restaurant,Pet Store,Gas Station
54,East Toronto,4,Coffee Shop,Pet Store,Fish Market,Bookstore,Gay Bar,Sandwich Place,Ice Cream Shop,Café,Distribution Center,Empanada Restaurant
